In [34]:
!pip install nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 18.3 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [35]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [36]:
df=pd.read_csv("/kaggle/input/full-netflix-dataset/data.csv")

In [37]:
df.isnull().sum()

url                   9158
title                  500
type                     0
genres                 148
releaseYear             12
imdbId                 682
imdbAverageRating      726
imdbNumVotes           726
availableCountries       0
dtype: int64

In [38]:
df=df.drop(columns=["url","imdbId"])


In [39]:
df

,title,type,genres,releaseYear,imdbAverageRating,imdbNumVotes,availableCountries
0,Forrest Gump,movie,"Drama, Romance",1994.0,8.8,2313221.0,MX
1,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,7.6,516523.0,"AT, CH, DE"
2,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,8.2,1220488.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM..."
3,Jarhead,movie,"Biography, Drama, War",2005.0,7.0,211314.0,"AD, AE, AG, AL, AO, AR, AT, AZ, BA, BB, BE, BG..."
4,Unforgiven,movie,"Drama, Western",1992.0,8.2,443310.0,"AU, BA, BE, BG, CZ, HR, HU, MD, ME, MK, NZ, PL..."
...,...,...,...,...,...,...,...
15855,NaN,tv,"Drama, Mystery",NaN,NaN,NaN,"AD, AE, AG, AL, AO, AR, AT, AU, AZ, BA, BB, BE..."
15856,NaN,tv,NaN,2024.0,NaN,NaN,"HK, ID, IN, KR, MY, PH, SG, TH"
15857,NaN,tv,NaN,2024.0,NaN,NaN,"AL, AO, AU, AZ, BA, BB, BG, BM, BS, BY, BZ, CA..."
15858,NaN,tv,NaN,NaN,NaN,NaN,"AG, AO, AR, AU, BB, BM, BO, BS, BZ, CA, CI, CL..."


In [40]:
df2=df.dropna(how="any",axis=0)
df2.isnull().sum()

title                 0
type                  0
genres                0
releaseYear           0
imdbAverageRating     0
imdbNumVotes          0
availableCountries    0
dtype: int64

In [41]:
df2

,title,type,genres,releaseYear,imdbAverageRating,imdbNumVotes,availableCountries
0,Forrest Gump,movie,"Drama, Romance",1994.0,8.8,2313221.0,MX
1,The Fifth Element,movie,"Action, Adventure, Sci-Fi",1997.0,7.6,516523.0,"AT, CH, DE"
2,Kill Bill: Vol. 1,movie,"Action, Crime, Thriller",2003.0,8.2,1220488.0,"AE, AL, AO, AT, AU, AZ, BG, BH, BY, CA, CI, CM..."
3,Jarhead,movie,"Biography, Drama, War",2005.0,7.0,211314.0,"AD, AE, AG, AL, AO, AR, AT, AZ, BA, BB, BE, BG..."
4,Unforgiven,movie,"Drama, Western",1992.0,8.2,443310.0,"AU, BA, BE, BG, CZ, HR, HU, MD, ME, MK, NZ, PL..."
...,...,...,...,...,...,...,...
15847,Mr. McMahon,tv,"Biography, Crime, Documentary",2024.0,7.7,6746.0,"AD, AE, AG, AL, AO, AR, AT, AU, AZ, BA, BB, BE..."
15848,Blood Legacy,tv,"Drama, Thriller",2024.0,7.4,101.0,"AD, AE, AG, AL, AO, AR, AT, AU, AZ, BA, BB, BE..."
15849,Chef's Table: Noodles,tv,Documentary,2024.0,7.5,201.0,"AD, AE, AL, AO, AR, AT, AU, AZ, BA, BB, BE, BG..."
15854,Making It in Marbella,tv,Reality-TV,2024.0,4.5,139.0,"AD, AE, AL, AO, AR, AT, AU, AZ, BA, BB, BE, BG..."


In [42]:
data=df2["genres"].str.split(',')
from sklearn.preprocessing import MultiLabelBinarizer
mult=MultiLabelBinarizer()
da=mult.fit_transform(data)
da2=pd.DataFrame(da,columns=mult.classes_)
da2

,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,Film-Noir,...,Mystery,Reality-TV,Romance,Sci-Fi,Short,Sport,Talk-Show,Thriller,War,Western
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15129,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15132,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [43]:
data1=df2["availableCountries"].str.split(',')
from sklearn.preprocessing import MultiLabelBinarizer
mult1=MultiLabelBinarizer()
da1=mult.fit_transform(data1)
da21=pd.DataFrame(da1,columns=mult.classes_)
da21

,AE,AG,AL,AO,AR,AT,AU,AZ,BA,BB,...,SK,SM,TC,TH,TN,TR,TW,TZ,UA,US
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15129,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
15130,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
15131,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
15132,1,0,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [44]:
df2.isnull().sum()

title                 0
type                  0
genres                0
releaseYear           0
imdbAverageRating     0
imdbNumVotes          0
availableCountries    0
dtype: int64

In [45]:
concaty=pd.concat([df2,da2,da21],axis=1).drop(columns=["genres","availableCountries"])
concaty=concaty.dropna(how='any',axis=0)
concaty

,title,type,releaseYear,imdbAverageRating,imdbNumVotes,Adventure,Animation,Biography,Comedy,Crime,...,SK,SM,TC,TH,TN,TR,TW,TZ,UA,US
0,Forrest Gump,movie,1994.0,8.8,2313221.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,The Fifth Element,movie,1997.0,7.6,516523.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Kill Bill: Vol. 1,movie,2003.0,8.2,1220488.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Jarhead,movie,2005.0,7.0,211314.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Unforgiven,movie,1992.0,8.2,443310.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15129,Zombieverse,tv,2023.0,4.4,1108.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15130,Vivant,tv,2023.0,7.0,638.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15131,Miniforce,tv,2014.0,5.0,95.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15132,Full Swing,tv,2023.0,8.0,4333.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
def text_preprocessing(text):
    if isinstance(text,str):
        data=text.lower()
    else:
        return ""
    data_re=re.sub(r"\W+"," ",data)
    data_tokenize=word_tokenize(data_re)
    tokens=[word for word in data_tokenize if word not in stopwords.words("english")]
    return " ".join(tokens)
def text_counterizer(text):
    vectorizer=CountVectorizer()
    data=text.apply(text_preprocessing)
    da=vectorizer.fit_transform(data)
    da=pd.DataFrame(da.toarray(),columns=vectorizer.get_feature_names_out())
    return da

In [47]:
da=text_counterizer(concaty["title"])

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/local/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/local/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
da

In [ ]:
concaty2=pd.concat([concaty,da],axis=1).drop(columns=["title"])
concaty2=concaty2.dropna(how='any',axis=0)
concaty2=pd.get_dummies(concaty2).astype(int)
concaty2

In [ ]:
X=concaty2.drop(columns=["imdbAverageRating"],axis=1)
Y=concaty2["imdbAverageRating"]


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
# allrandom_reg=RandomForestRegressor()
# random_reg.fit(X_train,Y_train)
# random_reg.score(X_test,Y_test)

In [ ]:
!pip install cirq

In [ ]:
import cirq

In [ ]:
X_train

In [ ]:
def quantum_layer(data):
    qubit=[cirq.GridQubit(0,i) for i in range(14)]
    circuit=cirq.Circuit()
    for i,value in enumerate(data):
        qubits=qubit[i%len(qubit)]
        circuit.append(cirq.H(qubits))
        circuit.append(cirq.rx(value*np.pi).on(qubits))
        circuit.append(cirq.X(qubits))
        circuit.append(cirq.ry(value*np.pi).on(qubits))
        circuit.append(cirq.Y(qubits))

        circuit.append(cirq.rz(value*np.pi).on(qubits))
        if i%len(qubit)<len(qubit)-1:
            circuit.append(cirq.CNOT(qubits,qubit[(i+1)%len(qubit)]))
        circuit.append(cirq.measure(qubits,key=f"qubit_{i}"))
        
    return circuit, qubit
X_train2 = np.random.rand(1700)  
circuit, qubit = quantum_layer(X_train2)
print(circuit)
# circuit,qubit=quantum_layer(X_train)

In [ ]:
!pip install tqdm

In [ ]:
from keras.layers import Layer,Dense
from keras.models import Sequential
import tensorflow as tf
from tqdm.keras import TqdmCallback

In [ ]:
def run_quantum_circuit(data):
    results=[]
    for inputs_data in data:
        circuit,qubit=quantum_layer(inputs_data)
        simulator=cirq.Simulator()
        result=simulator.run(circuit)
        measure0=[]
        for i in range(len(inputs_data)):
            measure0=result.measurements[f"qubit_{i}"].flatten()[0]
        results.append(measure0)
    return np.array(results,dtype=np.float32)
class QuantumLayer(Layer):
    def __init__(self):
        super(QuantumLayer,self).__init__()
    def call(self,inputs):
        output=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
        output.set_shape(inputs.shape)
        return output
model=Sequential([
    QuantumLayer(),
    Dense(1)
])
model.compile(optimizer='adam',loss="mse",metrics=['mae'])

In [ ]:
model.fit(X_train.iloc[:10,:],Y_train[:10],callbacks=[TqdmCallback(verbose=1)])

In [52]:
# !pip install qiskit[all]

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.1/28.1 MB 10.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.0/524.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 14.3 MB/s eta 0:00:00
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136818 sha256=b893135e91a58ce0c9b3fa240003a5dc3b12ef1a5b12455f123332b67623c2cf
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24059 sha256=ee6ba89f8a7668dba0ff4e868f9c7134aeea6cee1ed70f57d037626aaee074ba
  Stored in directory: /root/.cache/pip/wheels/2e/f2/2b/cb08b5fe129e4f69b7033061f256e5c551b0aa1160c2872aee
Successfully built pylatexe

In [53]:
# from qiskit import QuantumCircuit
# from qiskit import execute
# from qiskit.providers.ibmq import IBMQ
# def quantum_layer(data):
#     qubit=14
#     circuit=QuantumCircuit(14)
#     for i,value in enumerate(data):
#         qubits=i%qubit
#         circuit.h(qubits)
#         circuit.rx(value*np.pi,qubits)
#         circuit.x(qubits)
#         circuit.ry(value*np.pi,qubits)
#         circuit.y(qubits)

#         circuit.rz(value*np.pi,qubits)
#         if i<qubits-1:
#             circuit.cx(qubits,(qubits+1)%qubits)
#         circuit.measure_all()
        
#     return circuit

# def run_quantum_circuit(data): 
#     results=[]
#     for inputs_data in data:
#         circuit=quantum_layer(inputs_data)
#         provider=IBQM.get_provider().get_backend("ibmq_qasm_simulator")
#         jobs=execute(circuit,backend=provider,shots=1234)
#         results=jobs.results
#         counts=results.get_counts(circuit)
#         measure0=[]
#         for i in range(len(inputs_data)):
#             binary_representation=bit(i)[2:].zfill(14)
#             measure0=counts.get(binary_representation,0)
#         results.append(measure0)
#     return np.array(results,dtypes=np.float32)
# class QuantumLayer(Layer):
#     def __init__(self):
#         super(QuantumLayer,self).__init__()
#     def call(self,inputs):
#         output=tf.numpy_function(run_quantum_circuit,[inputs],tf.float32)
#         output.set_shape(inputs.shape)
#         return output
# model=Sequential([
#     QuantumLayer(),
#     Dense(1)
# ])
# model.compile(optimizer='adam',loss="mse",metrics=['mae'])


ImportError: cannot import name 'execute' from 'qiskit' (/usr/local/lib/python3.10/site-packages/qiskit/__init__.py)

In [61]:
# from qiskit import QuantumCircuit, Aer
# from qiskit.execute import execute
# from qiskit.providers.ibmq import IBMQ
